# Initialization & Inspection

In [1]:
# importing libraries
import numpy as np
import pandas as pd
import plotly.express as px
import math as mt
import requests 
from io import BytesIO

In [3]:
# trying to read in dataset from google sheet i made public and then loading it from the project directory if it fails

try:
    
    sheet_url = "https://docs.google.com/spreadsheets/d/1jzihXIadik_fkLF3u5dtNW2ELG0Ts3CNjOMFAVnrPL0/edit?usp=sharing"
    
    r = requests.get(sheet_url)
    
    df = pd.read_csv(BytesIO(r.content)) 
except:
    
    df = pd.read_csv("/Users/juansiliezar/sprint-4-software-development-tools/datasets/vehicles_us.csv")
    

In [27]:
# Printing the shape of the dataset
df.shape

(51525, 13)

In [26]:
# Printing a small sample of the data
df.sample(5)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
31824,7995,2008.0,ford escape,excellent,6.0,gas,65416.0,automatic,wagon,grey,1.0,2019-01-01,40
16619,7500,2009.0,honda accord,good,4.0,gas,NaN,automatic,sedan,grey,NaN,2018-08-16,21
12175,1,2015.0,honda cr-v,excellent,4.0,gas,17384.0,automatic,SUV,custom,NaN,2018-06-13,44
36493,19500,2015.0,nissan murano,excellent,6.0,gas,63837.0,automatic,SUV,NaN,1.0,2018-07-13,54
49363,11490,2016.0,ford focus,good,4.0,gas,61589.0,manual,hatchback,blue,NaN,2018-06-01,47


<div class='alert alert-info'> <b>Initial observations</b>

The dataset we are working with today is on used car listings posted online. There are 51,525 rows of data with each row corresponding to one used car for sale and 13 columns containg the following information about the vehicles:

1. price: asking price of the vehicle
2. model_year: year the vehicle came to market
3. model: model of the vehicle
4. condition: codition of the vehicle (e.g. new, good, fair, poor, etc) 
5. cylinders: the number of cylinders in the vehicle's engine
6. fuel: the type of fuel the vehicle runs on
7. odometer: the mileage on the vehicle
8. transmission: transmission type (e.g. manual, automatic)
9. type: vehicle type (e.g. truck, sedan, suv, etc.)
10. paint_color: vehicle color
11. is_4wd: whether the vehicle has 4-wheel drive or not
12. date_posted: the date the vehicle was listed for sale
13. days_listed: how many days has the car been listed


</div>

# Data Cleaning

Now I will:

1. Check for missing values and decide how to handle them
2. Check for duplicate entries
3. Verify and correct data types (if necessary)

In [16]:
# Checking for duplicate rows
df.duplicated().sum()

0

In [18]:
# Checking for missing values in model year column
missing_model_year = df[df['model_year'].isna()]
missing_model_year.shape[0]

3619

In [19]:
# Checking for missing values in is_4wd column
missing_4wd = df[df['is_4wd'].isna()]
missing_4wd.shape[0]

25953

In [20]:
# Investigating cylinders column
missing_cylinders = df[df['cylinders'].isna()]
missing_cylinders.shape[0]

5260

In [21]:
# Investigating paint_color column
missing_color = df[df['paint_color'].isna()]
missing_color.shape[0]

9267

In [22]:
# Investigating odometer column
missing_odometer = df[df['odometer'].isna()]
missing_odometer.shape[0]

7892

In [29]:
# Displaying data types
df.dtypes

price             int64
model_year      float64
model            object
condition        object
cylinders       float64
fuel             object
odometer        float64
transmission     object
type             object
paint_color      object
is_4wd          float64
date_posted      object
days_listed       int64
dtype: object

<div class='alert alert-info'>

### Here's what I found in the data cleaning stage

**Missing Values:**

- *model_year: 3,619 missing values*

- *cylinders: 5,260 missing values*

- *odometer: 7,892 missing values*

- *paint_color: 9,267 missing values*

- *is_4wd: 25,953 missing values (Note: This might be due to '0' or 'NaN' representing absence of 4-wheel drive)*


**Duplicate Entries:**

- *There were zero duplicate entries in the data*


**Data Types:**

- Most data types seem appropriate, but there are a few points to note:

    - *model_year, cylinders, and is_4wd are floats, which could be converted to integers. However, missing values may complicate this*

    - *date_posted should be converted to a datetime type to allow time series analysis*


**Action Steps:**

1. *Handle missing values: Depending on the context, we could fill them, drop them, or leave them as-is*

2. *Convert date_posted to datetime type*

3. *Consider converting model_year, cylinders, and is_4wd to integers, taking care of missing values*

</div>

In [30]:
# Correcting date posted dtype
df['date_posted'] = pd.to_datetime(df['date_posted'], format='%Y-%m-%d')

<div class='alert alert-info'>

Handling Missing Values:

1. **model_year, cylinders, & odometer:** Data could potentially be filled in by looking at vehicles of the same model and condition in the dataset, but I will leave them as NaN for now and re-evaluate this decision later in our analysis

2. **paint_color:** Since this is categorical I will categorize the missing values with the label  'unknown'

3. **is_4wd:** Assuming NaN or 0 indicates the absence of 4-wheel drive, I will replace NaN with '0' for clarity

</div>

In [31]:
# filling in missing values in paint color column
df['paint_color'] = df['paint_color'].fillna('unknown')

In [33]:
# filling in missing values in 4-wheel drive column
df['is_4wd'] = df['is_4wd'].fillna(0)

In [40]:
# confirming data was filled in as expected
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  int64         
 1   model_year    47906 non-null  float64       
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     46265 non-null  float64       
 5   fuel          51525 non-null  object        
 6   odometer      43633 non-null  float64       
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   51525 non-null  object        
 10  is_4wd        51525 non-null  float64       
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(2), object(6)
memory usage: 5.1+ MB


# Descriptive Statistics

Now that the data has been cleaned let's start exploring it

In [42]:
# displaying summary stats & including all columns, not just quantitative columns
df.describe(include='all')

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
count,51525.000000,47906.000000,51525,51525,46265.000000,51525,43633.000000,51525,51525,51525,51525.000000,51525,51525.00000
unique,NaN,NaN,100,6,NaN,5,NaN,3,13,13,NaN,NaN,NaN
top,NaN,NaN,ford f-150,excellent,NaN,gas,NaN,automatic,SUV,white,NaN,NaN,NaN
freq,NaN,NaN,2796,24773,NaN,47288,NaN,46902,12405,10029,NaN,NaN,NaN
mean,12132.464920,2009.750470,NaN,NaN,6.125235,NaN,115553.461738,NaN,NaN,NaN,0.496303,2018-10-25 01:57:46.270742528,39.55476
min,1.000000,1908.000000,NaN,NaN,3.000000,NaN,0.000000,NaN,NaN,NaN,0.000000,2018-05-01 00:00:00,0.00000
25%,5000.000000,2006.000000,NaN,NaN,4.000000,NaN,70000.000000,NaN,NaN,NaN,0.000000,2018-07-29 00:00:00,19.00000
50%,9000.000000,2011.000000,NaN,NaN,6.000000,NaN,113000.000000,NaN,NaN,NaN,0.000000,2018-10-25 00:00:00,33.00000
75%,16839.000000,2014.000000,NaN,NaN,8.000000,NaN,155000.000000,NaN,NaN,NaN,1.000000,2019-01-21 00:00:00,53.00000
max,375000.000000,2019.000000,NaN,NaN,12.000000,NaN,990000.000000,NaN,NaN,NaN,1.000000,2019-04-19 00:00:00,271.00000


<div class='alert alert-info'> 

#### Here's what I discovered from inspecting the dataset's summary stats

1. **price**: prices ranged from $1 - $375,000. the mean price was roughly $12,000

2. **model_year**: ranges from 1908 - 2019, however 75% of the vehicles came to market in 2006 or later

3. **model**: the most common model was the "ford f-150"

4. **condition**: most cars were said to be in excellent condition

5. **cylinders**: half of the vehicles have between 4 and 8 cylinders

6. **fuel**: the majority of vehicles were gas-powered

7. **odometer**: ranges from 0 - 990,000 miles. the mean mileage was roughly 115,000 miles

8. **transmission**: the most common transmission type was automatic

9. **type**: SUV's were the most common vehicle type

10. **paint_color**: the most common car color was white

11. **is_4wd**: about half of the vehicles have four-wheel drive

12. **date_posted**: ranges from 05/01/2018 - 04/19/2019

13. **days_listed**: values ranged from 0 - 271 with the mean days on market at roughly 40 days


</div>

# Visualizing the Data

Next I will use visualizations to further explore the data focusing on:

1. The distribution of key variables such as price, model year, odometer, & days listed

2. Possible relationships between key variables (price vs. odometer, price vs. model year)

In [43]:
# plotting the distribution of key variables with histograms

price_hist = px.histogram(df, x='price')
price_hist.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed